In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast, BertConfig, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.autograd

import torch.nn.functional as F
import torch
from torch import Tensor
from torch.nn.parameter import Parameter,   UninitializedParameter
from torch.nn import functional as F

In [2]:
def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)
    


def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3


def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

In [5]:
class MyLinearFunction(torch.autograd.Function):

        # Note that forward, setseup_context, and backward are @staticmethods
    @staticmethod
    def forward(ctx,input, weight, bias, grads, treshold):
        treshold = treshold
        if ((len(grads) <30 )):

            ctx.save_for_backward(input, weight, bias)
        else:
            u1, U, VT =Tucker_Decomposition(torch.cat(MyLayer.grads))
            ctx.save_for_backward(input,weight, bias, U, VT)
        ctx.size = input.shape[0]

        return  input @ weight.T  + bias
    


    @staticmethod
    def backward(ctx, grad_output):

        if len(ctx.saved_tensors) == 3:
#             print('before 30')
            input,weight, bias = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
#                 print('grad_output.T ', grad_output.T.shape)

                grad_weight =  torch.einsum('ijk, kjn ->kin',grad_output.T ,input)
                grad_weight = torch.sum(grad_weight, dim = 0)
                print(weight.shape, grad_weight.shape)
  
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        elif len(ctx.saved_tensors) == 5:

            input,weight, bias, U, VT = ctx.saved_tensors
            print(U)
            print(VT)
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
#                 print('grad_output.T ', grad_output.T.shape)
#                 print('input ', input.shape)
#                 grad_weight = grad_output.T @input 
                grad_weight =  torch.einsum('ijk, kjn ->kin',grad_output.T ,input)
                grad_weight = torch.sum(grad_weight, dim = 0)
                grad_weight = U @  grad_weight @ VT
                grad_weight = torch.where(torch.abs(grad_weight) >= treshold, grad_weight, 0)
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        return grad_input, grad_weight, grad_bias, None, None
        
        


In [6]:
class MyLinear(torch.nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.grads = []
        self.treshold = 1e-3
    def forward(self, x):
            return MyLinearFunction.apply(x, self.weight, self.bias, self.grads, self.treshold)
            

In [7]:
x= torch.randn((2,1,10))
a = torch.nn.Parameter(torch.randn((10,10)))
b = torch.nn.Parameter(torch.randn((10,10)))
c = torch.nn.Parameter(torch.randn((1, 10)))
u = torch.randn((10,10)) 
vt = torch.randn((10,10))
treshold = 1e-3

In [8]:
MyLayer = MyLinear(10, 10)
MyLayer.weight = nn.Parameter(a, requires_grad = True)
MyLayer.bias = nn.Parameter(c, requires_grad = True)

In [9]:
gt_layer = nn.Linear(10,10)
gt_layer.weight = torch.nn.Parameter(a)
gt_layer.bias = torch.nn.Parameter(c)
optimizer_gt = torch.optim.Adam(gt_layer.parameters())
criterion_gt = nn.MSELoss()

In [10]:
gt = torch.nn.Parameter(torch.randn((1, 10)))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(MyLayer.parameters())

In [11]:
x.shape

torch.Size([2, 1, 10])

In [12]:
gt_layer(x)

tensor([[[-0.9017,  3.2322,  0.1372, -1.5867,  0.8343, -2.5966, -0.2687,
          -0.5407, -0.0595,  6.2544]],

        [[ 2.1707, -5.5328, -5.2899,  1.3776,  5.5376,  2.0128,  1.0774,
          -0.8789,  4.8484, -4.2745]]], grad_fn=<ViewBackward0>)

In [13]:
MyLayer(x)

tensor([[[-0.9017,  3.2322,  0.1372, -1.5867,  0.8343, -2.5966, -0.2687,
          -0.5407, -0.0595,  6.2544]],

        [[ 2.1707, -5.5328, -5.2899,  1.3776,  5.5376,  2.0128,  1.0774,
          -0.8789,  4.8484, -4.2745]]], grad_fn=<MyLinearFunctionBackward>)

In [14]:
import numpy as np

In [15]:
for i in range(30):

    optimizer.zero_grad()
    optimizer_gt.zero_grad()
    
    output = MyLayer(x)
    w = gt_layer(x)
    assert np.allclose(output.detach().numpy(), w.detach().numpy())
    
    loss = criterion(output, gt)
    loss_gt = criterion_gt(w, gt)
    
    loss.backward()
    loss_gt.backward()
    
    print('torch \n', gt_layer.weight.grad.detach().numpy())
    print('my \n', MyLayer.weight.grad.detach().numpy())
    assert np.allclose(MyLayer.weight.grad.detach().numpy(), gt_layer.weight.grad.detach().numpy())
    
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

    optimizer.step()
    optimizer_gt.step()

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([2, 1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([10, 10]) torch.Size([10, 10])
torch 
 [[ 3.0146208e-01 -5.9559319e-02  8.4138095e-02  2.2165753e-01
   4.1548753e-01 -3.9088823e-02 -1.4523065e-01 -1.5270974e-01
   2.0339447e-01 -5.4777092e-01]
 [-7.4608803e-01  2.7459422e-02 -8.5602209e-02 -5.4330260e-01
  -8.7065619e-01  2.9995954e-01  2.2567123e-01  5.9890050e-01
  -5.5639678e-01  1.3665532e+00]
 [-8.1719679e-01  4.6113691e-01 -5.3447860e-01 -6.1404967e-01
  -1.5201479e+00 -4.0178955e-01  7.2789276e-01 -1.3811442e-01
  -4.1889420e-01  1.4577112e+00]
 [ 2.6447475e-01 -2.4486717e-02  4.5427807e-02  1.9324006e-01
   3.2802045e-01 -8.1334755e-02 -9.6448570e-02 -1.8512185e-01
   1.9071174e-01 -4.8308074e-01]
 [ 5.4000199e-01 -1.8933518e-01  2.3521420e-01  4.0068623e-01
   8.5287118e-01  7.0010081e-02 -3.5231623e-01 -1.2129208e-01
   3.2780471e-01 -9.7371507e-01]
 [ 3.9665496e-01 -1.9815031e-02  5.0843298e-02  2.8907433e-01
   4.6973658e-01 -1.5063445e-01 -1.2579447e-01 -3.0879375e-01
   2.9350063e-01 -7.2605002e-01]
 [ 5.721

/tmp/ipykernel_3066750/3824518368.py:31: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3571.)
  grad_weight =  torch.einsum('ijk, kjn ->kin',grad_output.T ,input)


In [16]:
optimizer.zero_grad()
optimizer_gt.zero_grad()

output = MyLayer(x)
w = gt_layer(x)
assert np.allclose(output.detach().numpy(), w.detach().numpy())

loss = criterion(output, gt)
loss_gt = criterion_gt(w, gt)

loss.backward()
loss_gt.backward()

#     print(i, gt_layer.weight.detach().numpy())
#     print(MyLayer.weight.grad.detach().numpy())
assert np.allclose(MyLayer.weight.grad.detach().numpy(), gt_layer.weight.grad.detach().numpy())

MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

optimizer.step()
optimizer_gt.step()

tensor([[-1.5948e-01,  3.8938e-02, -5.3494e-01,  2.3804e-01,  4.7768e-02,
         -3.6613e-01,  5.8846e-01,  3.3953e-01, -1.7935e-01, -1.2545e-02],
        [ 4.1502e-01, -3.9455e-01, -1.5685e-01,  3.3412e-03,  3.7345e-02,
          1.9950e-02,  9.9595e-02,  1.8330e-01,  6.8596e-01, -3.6280e-01],
        [ 5.1500e-01,  6.6940e-01, -2.1380e-01,  2.3421e-01, -9.2638e-02,
         -1.8506e-01, -3.5338e-01,  9.0308e-02,  7.6115e-03, -1.0096e-01],
        [-1.3428e-01,  1.0380e-01, -5.1928e-01,  7.1269e-02, -7.4149e-02,
          8.2932e-01, -4.3440e-02,  3.6235e-02, -1.3192e-03,  5.2512e-04],
        [-3.1524e-01, -1.4233e-01, -2.2747e-01,  1.9467e-01, -5.0547e-02,
         -2.2693e-01, -4.5131e-01,  1.5222e-01,  4.0215e-01,  5.9000e-01],
        [-2.1186e-01,  1.9316e-01, -3.0211e-01, -5.2092e-02,  2.3875e-01,
         -1.8239e-01,  8.3420e-02, -7.9010e-01,  2.8616e-01, -1.4298e-01],
        [-1.9218e-02,  1.9092e-01, -1.1742e-01, -6.2109e-01,  6.6372e-01,
         -7.8722e-03, -1.1122e-0

AssertionError: 

In [17]:
U_ = torch.tensor([[-1.5948e-01,  3.8938e-02, -5.3494e-01,  2.3804e-01,  4.7768e-02,
         -3.6613e-01,  5.8846e-01,  3.3953e-01, -1.7935e-01, -1.2545e-02],
        [ 4.1502e-01, -3.9455e-01, -1.5685e-01,  3.3412e-03,  3.7345e-02,
          1.9950e-02,  9.9595e-02,  1.8330e-01,  6.8596e-01, -3.6280e-01],
        [ 5.1500e-01,  6.6940e-01, -2.1380e-01,  2.3421e-01, -9.2638e-02,
         -1.8506e-01, -3.5338e-01,  9.0308e-02,  7.6115e-03, -1.0096e-01],
        [-1.3428e-01,  1.0380e-01, -5.1928e-01,  7.1269e-02, -7.4149e-02,
          8.2932e-01, -4.3440e-02,  3.6235e-02, -1.3192e-03,  5.2512e-04],
        [-3.1524e-01, -1.4233e-01, -2.2747e-01,  1.9467e-01, -5.0547e-02,
         -2.2693e-01, -4.5131e-01,  1.5222e-01,  4.0215e-01,  5.9000e-01],
        [-2.1186e-01,  1.9316e-01, -3.0211e-01, -5.2092e-02,  2.3875e-01,
         -1.8239e-01,  8.3420e-02, -7.9010e-01,  2.8616e-01, -1.4298e-01],
        [-1.9218e-02,  1.9092e-01, -1.1742e-01, -6.2109e-01,  6.6372e-01,
         -7.8722e-03, -1.1122e-01,  3.2566e-01,  2.4433e-02,  6.3525e-02],
        [ 2.9517e-03, -1.2796e-02,  2.6107e-01,  6.7130e-01,  6.7274e-01,
          1.6763e-01,  7.1977e-03,  1.7383e-02,  3.7121e-04,  5.2901e-04],
        [-3.5336e-01, -2.3130e-01, -1.6054e-01,  7.6900e-02,  2.7097e-02,
         -1.6498e-01, -5.1743e-01,  1.1264e-01, -2.5030e-01, -6.4725e-01],
        [ 4.9956e-01, -4.8606e-01, -3.5092e-01, -2.1793e-02,  1.6987e-01,
         -5.8305e-02, -1.6199e-01, -2.7418e-01, -4.3632e-01,  2.5791e-01]])
VT_= torch.tensor([[-3.5454e-01, -6.6654e-02, -4.0976e-01, -6.3395e-04, -7.5717e-01,
         -3.1129e-01, -1.1695e-01,  9.2357e-03, -1.2332e-01, -5.2644e-02],
        [ 8.6041e-02, -2.8406e-01,  9.4606e-02, -7.7726e-01, -1.8160e-01,
          3.6028e-01, -6.5165e-02, -9.8216e-02, -2.4891e-01,  2.4535e-01],
        [-1.1531e-01,  2.8528e-01, -1.1984e-01, -8.2185e-02, -8.4721e-02,
          5.3313e-02,  1.3928e-02, -9.7981e-02,  6.6256e-01,  6.5222e-01],
        [-2.6139e-01, -3.5932e-02,  2.9844e-01, -1.9225e-01, -6.4144e-03,
         -3.7131e-01,  7.9650e-01, -1.7866e-01, -1.0187e-02, -3.8687e-03],
        [-5.0967e-01,  2.9876e-01,  4.1062e-01,  2.5528e-01, -2.2195e-01,
          5.8072e-01,  1.0738e-02, -4.4162e-02, -1.6720e-01, -2.6475e-02],
        [ 1.8871e-02,  5.1223e-01, -5.9361e-02, -4.6016e-01, -4.2670e-02,
          4.5043e-02, -4.5310e-02, -3.3670e-02,  3.2940e-01, -6.3754e-01],
        [ 1.8864e-01, -2.9935e-01, -2.6105e-01,  2.2712e-01, -1.2299e-01,
          3.4891e-01,  2.0757e-01, -6.9849e-01,  2.0268e-01, -2.1482e-01],
        [ 1.5013e-01,  5.8131e-01, -1.9875e-01, -1.2520e-02,  1.1942e-01,
         -1.5355e-01, -3.5369e-02, -4.7262e-01, -5.3224e-01,  2.3268e-01],
        [-2.3214e-01, -1.7635e-01,  4.6013e-01, -5.1192e-02,  8.6751e-02,
         -3.7077e-01, -5.4778e-01, -4.8289e-01,  1.4155e-01, -4.8881e-02],
        [ 6.4277e-01,  1.4806e-01,  4.7773e-01,  1.4448e-01, -5.4634e-01,
         -1.0266e-01,  2.9973e-02,  3.4970e-02,  5.3719e-02,  4.2318e-02]])

In [18]:
assert np.allclose(MyLayer.weight.grad.detach().numpy(), U_.detach().numpy()@ gt_layer.weight.grad.detach().numpy() @ VT_.detach().numpy())

AssertionError: 

In [19]:
gradd =  U_.detach().numpy()@ gt_layer.weight.grad.detach().numpy() @ VT_.detach().numpy()


In [20]:
gradd = np.where(np.abs(gradd) >= 1e-3, gradd, 0)

In [21]:
assert np.allclose(MyLayer.weight.grad.detach().numpy(),gradd)

AssertionError: 

In [22]:
MyLayer.weight.grad.detach().numpy()

array([[-0.7073008 ,  0.35136864,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-3.2391262 , -0.1361398 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.36155623,  0.49512938,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.5647041 ,  0.28084785,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.306384  ,  0.7255074 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.0894037 ,  0.42487943,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.35963014,  0.45247388,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

In [23]:
gradd

array([[-0.70729345,  0.35137084,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-3.2391415 , -0.13613176,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.3615569 ,  0.49512887,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.5647188 ,  0.28085506,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.30637127,  0.7255081 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.0894086 ,  0.42488298,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.3596248 ,  0.4524713 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

In [8]:
for i in range(30):
    optimizer.zero_grad()
    output = MyLayer(x)
    loss = criterion(output, gt)
  
    loss.backward()
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

    optimizer.step()

In [9]:
optimizer.zero_grad()
output = MyLayer(x)
loss = criterion(output, gt)

loss.backward()

optimizer.step()